In [201]:
import numpy as np
from numba import jit
import matplotlib.pyplot as plt


@jit(nopython=True)
def couplingDistribution(disttype='uniform',a=1,b=0):
    """
    Returns samples from specified distribution
    """
    
    if disttype=='uniform':
        return a*(2*np.random.rand()-1) + b
    if disttype=='gaussian':
        return np.random.normal(b,a)
    if disttype=='bernoulli':
        return a*(2*np.random.choice(2) - 1) + b
    if disttype=='multinoulli':
        return 0.1*(np.random.choice(5) - 3)

@jit(nopython=True)
def makeRandomCouplingMatrix(nspins,sparsity,scale=1.0,disttype='uniform',a=1,b=0):
    """
    Creates random Ising model coupling matrix with specified sparsity 
    """
    
    J = np.zeros((nspins,nspins))
    numEntries = int(sparsity*J.size)
    numFilled = 0
    while numFilled < numEntries:
            m,n = np.random.choice(nspins,2)

            if J[m,n]==0 and J[n,m]==0 and m!=n:
                    J[m,n] = scale*couplingDistribution(disttype,a,b)
                    J[n,m] = J[m,n]
                    numFilled += 2
                    
            if J[m,n]==0 and m==n:
                J[m,n] = scale*couplingDistribution(disttype,a,b)
                numFilled += 1

    return J


@jit(nopython=True)
def conditionalIsingProb(i,x,J,nspins,beta=1,xfmt=0):
    """
    Returns the conditional probability vector of spin i
    given values for all other spins ~i
    """
    pxi = np.zeros(2)
    
    # 0 --> -1, 1 --> 1
    for orientation in range(2):
        
        # Add single site term to energy
        pxi[orientation] = 2*J[i,i]*int(2*orientation - 1)
        
        # Add coupling terms to energy
        for spin in range(nspins):
            if spin!=i:
                pxi[orientation] += (J[i,spin] + J[spin,i])*int(2*orientation - 1)*x[spin]
                
    #pxi = np.exp(beta*pxi)
    pxi = np.exp(beta*pxi/2)
    pxi = pxi/np.sum(pxi)
    
    return pxi 


@jit(nopython=True)
def GibbsSample(nsamples,J,nspins,beta=1,burn=0):
    """
    Gibbs sample the Ising model specified by coupling matrix
    J at inverse temperature beta. Discards the first burn
    samples.
    Output: nsamples x nspins matrix
    """
    
    samples = np.empty((nsamples,nspins),dtype=np.int64)
    xj = np.random.choice(2,nspins) #Random initial state
    xj = 2*xj - 1
    
    j = 0
    
    while j<nsamples+burn:
        
        
        for i in range(nspins):
        
            pxi = conditionalIsingProb(i,xj,J,nspins,beta)
            pxi = np.cumsum(pxi)
            r = np.random.rand()
            
            for orientation in range(2):
                if r <= pxi[orientation]:
                    xj[i] = int(2*orientation - 1)
                    break
        
        if j>=burn:
            samples[j-burn,:] = xj[:]
        
        j += 1
    
    return samples

@jit(nopython=True)
def fullIsingProb(J,nspins,beta=1):
    """
    Returns the full probability distribution for an
    Ising model with specified J matrix
    """
    nstates = 2**nspins
    probabilities = np.zeros(nstates)
    
    config = np.zeros(nspins,dtype=np.int64)
    for state in range(nstates):
        
        if state==0:
            config = -1*np.ones(nspins,dtype=np.int64)
        else:
            num = state
            for spin in range(nspins):
                config[nspins-1-spin] = 2*int(num%2) - 1
                num //= 2
        
        
#         for spin1 in range(nspins):
#             for spin2 in range(spin1,nspins):
#                 probabilities[state] += J[spin1,spin2,config[spin1],config[spin2]]

#         probabilities[state] = np.exp(beta*probabilities[state])
        
        
        for spin1 in range(nspins):
            probabilities[state] += 2*J[spin1,spin1]*config[spin1]
            for spin2 in range(nspins):
                if spin1!=spin2:
                    probabilities[state] += J[spin1,spin2]*config[spin1]*config[spin2]
        
        probabilities[state] = np.exp(beta*probabilities[state]/2)
    
    probabilities = probabilities/np.sum(probabilities)
        
    return probabilities

In [206]:
nspins = 4
nsamples = 1000
sparsity = 0.5


J = makeRandomCouplingMatrix(nspins,sparsity,scale=0.5,disttype='bernoulli')
samples = GibbsSample(nsamples,J,nspins,beta=1,burn=1000)

#np.save("gibbssamples.npy",samples)
np.save("isingcoupling.npy",J)

samples_csv = np.insert(samples,0,np.ones(nsamples),axis=1)
np.savetxt('isingsamples.csv',samples_csv,delimiter=",",fmt='%d')

In [207]:
actualProb = fullIsingProb(J,nspins,beta=1)

sampledProb = np.zeros(int(2**nspins))
for sample in samples:
    index = 0
    for j in range(nspins):
        index += ((sample[j]+1)//2)*2**j
    sampledProb[index] += 1
sampledProb = sampledProb/np.sum(sampledProb)

print("J = \n",J)
print("Actual = ",np.around(np.sort(actualProb),3))
print("Sampled = ",np.around(np.sort(sampledProb),3))

J = 
 [[ 0.   0.5  0.5  0. ]
 [ 0.5 -0.5  0.5  0.5]
 [ 0.5  0.5  0.   0. ]
 [ 0.   0.5  0.   0. ]]
Actual =  [0.007 0.007 0.007 0.019 0.019 0.019 0.019 0.019 0.019 0.051 0.051 0.051
 0.051 0.14  0.14  0.38 ]
Sampled =  [0.005 0.006 0.007 0.012 0.018 0.018 0.019 0.02  0.03  0.047 0.049 0.053
 0.054 0.121 0.147 0.394]
